In [1]:
products.GetType()

Detenido debido a un error.


Error: input.fsx (1,1)-(1,9) typecheck error El valor, espacio de nombres, tipo o módulo "products" no está definido.

Método que te permite saber el tipo de dato
El error en el código es porque el último dato no tiene la estructura (eat, drink). Le falta el último drink
La ventaja del tipo de dato option es la cantidad de funciones que tiene similares al tipo result

In [2]:
Seq.choose

Aplica una función a cada elemento de la secuencia y solo devuelve los del tipo some
id es la identidad

¿Qué ventaja tiene usar el dato option?
Este tipo de datos nos permite no trabajar con "nros mágicos". Por ejemplo, si estoy midiendo algo y de repende deja de medir, no tengo que poner un "0" o un "-1" en la cadena



### Nutrientes en la alimentación

> Haga el ejercicio de producir el código **sin mirar el archivo de datos!!** 

El archivo [food.csv]() extraído de [acá](https://data.nal.usda.gov/) contiene datos de nutrientes de una gran cantidad de productos de alimentación. Los datos corresponden en cada caso a 100 g de cada 
producto alimenticio. Cada fila del archivo corresponde a un producto distinto.


- ¿Cuántas categorías de nutrientes hay?

A partir de la respuesta anterior, diseñe el código de forma tal que se pueda acceder a los datos de nutrientes _sin necesidad de usar índices_. 
>> Ayuda: los diccionarios `Map...` pueden ser sus aliados.


- Valide la estructura de la información: para ello compruebe que cada una de las filas de datos tiene la información de todos los nutrientes
- Encuentre los diez alimentos con más contenido de colesterol, y los 10 con menos contenido de colesterol.
- Encuentre los alimentos que contienen pan (_bread_) y muestre su contenido de carbohidratos en una tabla
- Entre los productos con leche, encuentre aquellos que tienen el menor contenido de grasas. 

### La máquina expendedora, toma 32768...

Para completar el modelo de la máquina expendedora, necesitamos definir los inputs y outputs. Asimismo, el fabricante de la máquina expendedora nos informa que a lo sumo habrá 7 estantes, y a lo sumo 10 productos por estante. 
La máquina acepta la máquina recibe monedas de 1 doblón (equivalente a 100 centavos), 50, 25 y 10
centavos de doblón; o en tarjeta de crédito. El vuelto (de existir) también se genera en monedas.

#### Inputs

Usando el modelo que diseño en [el ejercicio de monedas](https://ibprogramacionavanzada.github.io/fsharp/14_Exercises/#monedas). 

- Extienda los tipos de datos para agregar la posibilidad de introducir monedas de 1 doblón. 

Además, el usuario debe seleccionar el producto, con una letra (correspondiente al estante), y un
número (correspondiente al producto en el estante). 

- Cree los tipos de datos para modelar este input.
- Cree una función que valide dicho input, de acuerdo a la configuración de número de estantes y número de productos por estante.
- Cree una función que de aviso al cliente de cuánto dinero falta introducir, en caso en que
  el dinero introducido no ha sido suficiente.
  
#### Outputs  

Cree las funciones que entregan el producto al usuario (descontándolo adecuadamente del stock correspondiente), y el vuelto, si lo hubiere. 

#### Extra: un caso de borde

Un caso raro, pero que podría ocurrir, es el siguiente: el usuario introduce las monedas, suficientes
como para comprar el producto que quiere, pero la máquina no tiene monedas suficientes para dar 
el vuelto.



In [10]:
// let test = 
//     let a = [1 ; 2 ; 3]
//     try
//         List.find (fun b -> b=4) a
//     with
//         | :? System.Collections.Generic.KeyNotFoundException as ex  -> 
//             printfn "No hay ningún producto en tal lugar"
//             -1 

// let c = test
// printfn "%A"

-1


In [46]:
type monedas = 
    {
        M10: int
        M25: int
        M50: int
        M100:int //1 doblon = 100 centavos
    }

let dinero (monedas:monedas) =
    (float monedas.M10)*0.1 + (float monedas.M25)*0.25 + (float monedas.M50)*0.5 + (float monedas.M100)*1.


type categoria =
    | Chocolate
    | PapasFritas
    | Bebida
    | Galletitas
    | Vacio

type producto = 
    {
        nombre: string
        categoria: categoria
        precio: float
    }

type lugar = 
    {
        producto: producto
        cantidad: int
        estante: string
        posicion: int
    }

let producto_fantasma =
    {
        nombre = "fantasma"
        categoria = Vacio
        precio = 0.
    }

let lugar_fantasma =
    {
        producto = producto_fantasma
        cantidad = 0
        estante = "h"
        posicion = -1
    }

type maquina = 
    {
        lugares: lugar list
    }

let find_lugar (maquina:maquina) (estante:string) (posicion:int)= 
    try
        List.find(fun lugar -> lugar.estante = estante && lugar.posicion = posicion) maquina.lugares
    with
        | :? System.Collections.Generic.KeyNotFoundException as ex  -> 
            // printfn "No hay ningún producto en tal lugar"
            lugar_fantasma


// exception ExceptionForLugar of string 

// let validar_lugar (maquina:maquina) (estante:string) (posicion:int) =
    // try
    //     find_lugar (maquina:maquina) (estante:string) (posicion:int)
    // with
    // | :? System.InvalidOperationException  -> 
    //     printfn "Lugar no encontrado" 
    
// let verifico_precio (dinero_usuario) (precio_producto) =
//     if dinero_usuario > precio_producto then
//         dinero_usuario
//     else
//         printfn "Saldo insuficiente"
//         -1


let verifico_lugar (lugar:lugar) =
    if lugar = lugar_fantasma then
        printfn "No hay ningún producto en tal posición"
        false
    else
        true

let verifico_saldo (precio:float) (dinero_usuario: float) =
    let diferencia = precio - dinero_usuario
    if diferencia <= 0 then
        true
    else
        printfn "Saldo insuficiente, falta introducir $ %f"  diferencia
        false

let comprar (maquina:maquina) (estante:string) (posicion: int) (monedas:monedas) = 
    //Verifico si en esa posicion hay al menos un producto
    let lugar = find_lugar maquina estante posicion
    if (verifico_lugar lugar) = true && (verifico_saldo lugar.producto.precio (dinero monedas)) = true then

        //Cambio el nro de productos
        let lugar_new:lugar = 
            {
                producto = lugar.producto
                cantidad = lugar.cantidad - 1
                estante = lugar.estante
                posicion = lugar.posicion
            }
        printfn "Se compró el producto %A" lugar_new.producto.nombre
        let vuelto = (dinero monedas) - lugar.producto.precio
        printfn "Tiene un vuelto de %A" vuelto
        //Devuelvo la maquina
        {
            lugares = (List.filter (fun x -> x <> lugar) maquina.lugares) @ [lugar_new]
        }
    else
        // printf "No se compró el producto %A" lugar.producto.nombre
        maquina
        

let reponer (maquina:maquina) (estante:string) (posicion:int) =
    //Llevo a 15 el producto de tal posición.
    let lugar = find_lugar maquina estante posicion
    if lugar.cantidad < 15 then
        let lugar_new:lugar = 
            {
                producto = lugar.producto
                cantidad = 15
                estante = lugar.estante
                posicion = lugar.posicion
            }
        printf "Se compró el producto %A" lugar_new.producto.nombre
        //Devuelvo la maquina
        {
            lugares = (List.filter (fun x -> x <> lugar) maquina.lugares) @ [lugar_new]
        }

    else
        maquina


In [47]:
//Defino un conjunto de productos y sus lugares correspondientes

let Rapanui:producto = {nombre = "Chocolate Rapanui"; categoria = Chocolate; precio = 1.}
let Lays:producto = {nombre = "Lays"; categoria = PapasFritas; precio = 0.5}
let CocaCola:producto = {nombre = "Coca Cola 2L"; categoria = Bebida; precio = 1.}
let Rumba:producto = {nombre = "Rumba"; categoria = Galletitas; precio = 0.25}

//Lugares genéricos. Podría inicializar varios lugares del mismo producto con distinta cantidad pero no lo hago por simplicidad
let Lugar_Rapanui:lugar = {producto = Rapanui; cantidad = 10; estante = "a"; posicion = 0}
let Lugar_Lays:lugar = {producto = Lays; cantidad = 15; estante = "b"; posicion = 0}
let Lugar_CocaCola:lugar = {producto = CocaCola; cantidad = 1; estante = "c"; posicion = 0}
let Lugar_Rumba:lugar = {producto = Rumba; cantidad = 8; estante = "d"; posicion = 0}

//Listas de lugares
let maquina:maquina = {lugares = [Lugar_Rapanui; Lugar_Lays; Lugar_CocaCola; Lugar_Rumba]}

let dinero_usuario:monedas = 
    {
        M10 = 0
        M25 = 1
        M50 = 0
        M100 = 1
    }

printf "%A" (comprar maquina "a" 0 dinero_usuario)

Se compró el producto "Chocolate Rapanui"
Tiene un vuelto de 0.25
{ lugares =
   [{ producto = { nombre = "Lays"
                   categoria = PapasFritas
                   precio = 0.5 }
      cantidad = 15
      estante = "b"
      posicion = 0 }; { producto = { nombre = "Coca Cola 2L"
                                     categoria = Bebida
                                     precio = 1.0 }
                        cantidad = 1
                        estante = "c"
                        posicion = 0 }; { producto = { nombre = "Rumba"
                                                       categoria = Galletitas
                                                       precio = 0.25 }
                                          cantidad = 8
                                          estante = "d"
                                          posicion = 0 };
    { producto = { nombre = "Chocolate Rapanui"
                   categoria = Chocolate
                   precio = 1.0 }
      cantida

In [4]:
type Monedas = 
    {
        M100:int //1 doblon = 100 centavos
        M10: int
        M25: int
        M50: int
    }

type Categoria =
    | Chocolate
    | PapasFritas
    | Bebida
    | Galletitas

type Producto = 
    {
        nombre: string
        categoria: Categoria
        precio: float
    }

type Lugar = 
    {
        estante: string
        posicion: int
    }


type Producto_en_Maquina = 
    {
        lugar: Lugar
        cantidad: int
        producto: Producto
    }


let verifico_lugar lugar:Lugar = 
    if lugar.posicion >= 10 then //Las posiciones se cuentan desde 0
        false
    else


let input (dinero:Monedas) (estante: string) (posicion: int) =




let Cantidad (dinero: Monedas) =
    0.10*(float dinero.M10) + 0.25*(float dinero.M25) + 0.50*(float dinero.M50)

printf "%A" (Cantidad test)

Detenido debido a un error.


Error: input.fsx (43,1)-(43,4) parse warning Posible sangría incorrecta: este token está fuera del contexto iniciado en la posición (38:5). Pruebe aplicando más sangría a este token o usando convenciones de formato estándar.
input.fsx (43,1)-(43,4) parse warning Posible sangría incorrecta: este token está fuera del contexto iniciado en la posición (38:5). Pruebe aplicando más sangría a este token o usando convenciones de formato estándar.
input.fsx (48,1)-(48,4) parse warning Posible sangría incorrecta: este token está fuera del contexto iniciado en la posición (43:1). Pruebe aplicando más sangría a este token o usando convenciones de formato estándar.
input.fsx (48,1)-(48,4) parse warning Posible sangría incorrecta: este token está fuera del contexto iniciado en la posición (43:1). Pruebe aplicando más sangría a este token o usando convenciones de formato estándar.
input.fsx (51,1)-(51,7) parse warning Posible sangría incorrecta: este token está fuera del contexto iniciado en la posic